In [127]:
import os
import itertools

import pandas as pd
import xarray as xr

import climatools.lblnew.pipeline as pipeline
import climatools.lblnew.setup_bestfit as setup_bestfit
import climatools.lblnew.bestfit_params as bestfit



In [164]:
def fpath_ktable():
    molecule = 'h2o'
    band = '1'
    dv = .005
    nv = 200
    commitnumber = '9d81085'
    
    atmpro = 'mls'
    tsfc = 294
    
    param = bestfit.kdist_params(molecule='h2o', band='1')
    param['molecule'] = molecule
    param['band'] = band
    param['dv'] = dv
    param['nv'] = nv
    param['commitnumber'] = commitnumber
    param['atmpro'] = atmpro
    param['tsfc'] = tsfc
    
    fpath = os.path.join(pipeline.get_dir_case(param, 
                                               setup=setup_bestfit),
                         'ktable.dat')
    return fpath



def load_ktable(fpath):
    '''
    Returns k-table in the same format as in lblnew.f
    
    Parameter
    ---------
    fpath: string
        Path to the file 'ktable.dat', generated by "lblnew.f".
    ktable: array (number of (p, t) pairs, number of g-intervals)
        Absorption coefficient values calculated at selected 
        (pressure, temperature) pairs.
    '''
    df = pd.read_csv(fpath, sep=r'\s+')
    df = df.set_index('g')
    ng = len(df.index.value_counts())
    ktable = df['k'].values.reshape(ng, -1).T
    return ktable



def into_chunks(l, chunksize):
    return itertools.zip_longest(*(chunksize * [iter(l)]))



def vector_to_F77list(array, num_values_per_line=4):
    '''
    '''
    strfmt = '{:15.6e}'
    
    chunks = into_chunks(array, num_values_per_line)
    
    chunks = list(chunks)
    
    lines = []
    for chunk in chunks[:-1]:
        vs = [strfmt.format(v) for v in chunk if v != None]
        line = ','.join(vs)
        line = line + ','
        lines.append(line)
        
    vs = [strfmt.format(v) for v in chunks[-1] if v != None]
    line = ','.join(vs)
    lines.append(line)
    
    return lines
    

    
def ktable_to_F77(ktable):
    npt, ng = ktable.shape
    
    num_values_per_line = 4
    
    last_line = '/)'
    
    lines = []
    for ig in range(ng):
        first_line = 'ktable(:, {}) = (/'.format(ig + 1)
        
        lines_ig = vector_to_F77list(ktable[:, ig], 
                                 num_values_per_line=num_values_per_line)
        
        lines_ig = [first_line] + lines_ig + [last_line]
        
        lines_ig_amp = []
        lines_ig_amp.append(6 * ' ' + lines_ig[0])
        for l in lines_ig[1:]:
            lines_ig_amp.append(5 * ' ' + '&' + l)
            
        lines.extend(lines_ig_amp)
    
        chunks = into_chunks(ktable[:, ig], num_values_per_line)

    fortran = '\n'.join(lines)
    return fortran



def save_ktable(filename, s):
    clirad_dir = os.path.join('/chia_cluster/home/jackyu/radiation',
                              'clirad-lw/LW/lee_hitran2012_update/')
    
    with open(os.path.join(clirad_dir, filename),
              mode='w', encoding='utf-8') as f:
        f.write(s)
        
        
        
def script():
    ktable = load_ktable(fpath_ktable())

    save_ktable('ktable_h2o_band01.f', ktable_to_F77(ktable))
    

In [165]:
script()

In [135]:
itertools.zip_longest?